In [1]:
%matplotlib inline

import os, sys, time
import pandas as pd
import numpy as np
from os.path import join

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions

data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'


start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

In [2]:
nrows = 10000
bin_num = 1 ## number of bins to separate data by start_time
tmp_train, tmp_test, bins, bin_names = bosch_functions.create_grouped_index_df(bin_num)

data loading takes  61.9  seconds.


In [3]:
## create the skipped row numbers
none_selected_window_num = [np.NaN]
skipped_test_row_num = tmp_test.loc[tmp_test['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()
skipped_train_row_num = tmp_train.loc[tmp_train['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()

train_cat_cols  = pd.read_csv(join(data_path, train_cat_file), index_col=id_column_name, nrows=0)
train_date_cols = pd.read_csv(join(data_path, train_date_file), index_col=id_column_name, nrows=0)
train_num_cols  = pd.read_csv(join(data_path, train_num_file), index_col=id_column_name, nrows=0)

### section to obtain the feature impotance from xgboost model
#bin_0_data_path = '/home/ymm/kaggle/bosch/data_2_bins_xgb_combined_models/data_bin_0_models'
#bin_1_data_path = '/home/ymm/kaggle/bosch/data_2_bins_xgb_combined_models/data_bin_1_models'
#bin_0_selected_col_names = collect_feature_names(bin_0_data_path, 'feature', 'fscore', 10)
#bin_1_selected_col_names = collect_feature_names(bin_1_data_path, 'feature', 'fscore', 10)

regular_data_path = '/home/ymm/kaggle/bosch/data_1_bins_cross_fit_xgb_models/data_bin_0_models'

In [24]:
def collect_feature_names(data_path, fea_name='feature', thres_name = None, thres = 10):
    csv_files = [f for f in os.listdir(data_path) if '.csv' in f]
    feature_names = set()
    for file_name in csv_files:
        data = pd.read_csv(join(data_path, file_name), index_col=0)
        if thres_name is None:
            feature_names = feature_names.union(data[fea_name])
        else:
            feature_names = feature_names.union(data.loc[data[thres_name] > thres, fea_name])

    return feature_names

## collect feature names based on the fscore

bin_regular_selected_col_name = collect_feature_names(regular_data_path, 'feature', 'fscore', 10)

selected_cat_col_names =  train_cat_cols.columns[train_cat_cols.columns.isin(bin_regular_selected_col_name)].tolist()
selected_num_col_names =  train_num_cols.columns[train_num_cols.columns.isin(bin_regular_selected_col_name)].tolist()
selected_dat_col_names =  train_date_cols.columns[train_date_cols.columns.isin(bin_regular_selected_col_name)].tolist()
test_num_col_names     =  selected_num_col_names[:]
selected_dat_col_names.extend([id_column_name, start_time_column_name])
selected_cat_col_names.extend([id_column_name])
selected_num_col_names.extend([id_column_name, dep_var_name])
test_num_col_names.extend([id_column_name])

In [29]:
print len(test_num_col_names), len(selected_num_col_names)

292 293


In [30]:
start_time = time.time()
train_cat  = pd.read_csv(join(data_path, train_cat_file),   index_col='Id', skiprows=skipped_train_row_num, usecols=selected_cat_col_names)
test_cat   = pd.read_csv(join(data_path, test_cat_file),    index_col='Id', skiprows=skipped_test_row_num,  usecols=selected_cat_col_names)
train_dat  = pd.read_csv(join(data_path, train_date_file),  index_col='Id', skiprows=skipped_train_row_num, usecols=selected_dat_col_names)
test_dat   = pd.read_csv(join(data_path, test_date_file),   index_col='Id', skiprows=skipped_test_row_num,  usecols=selected_dat_col_names)
train_num  = pd.read_csv(join(data_path, train_num_file),   index_col='Id', skiprows=skipped_train_row_num, usecols=selected_num_col_names)
test_num   = pd.read_csv(join(data_path, test_num_file),    index_col='Id', skiprows=skipped_test_row_num,  usecols=test_num_col_names)

print 'finish reading data by columns selected using xgboost feature importance, using {} seconds.'.format(round(time.time() - start_time, 2))

finish reading data by columns selected using xgboost feature importance, using 152.64 seconds.


In [21]:
len(selected_num_col_names)

292

In [31]:
print train_cat.shape, train_num.shape, train_dat.shape

(673861, 12) (673861, 292) (673861, 67)


In [32]:
print test_cat.shape, test_num.shape, test_dat.shape

(674503, 12) (674503, 291) (674503, 67)


### process the date features

In [8]:
tmp_train_date = train_dat.copy()

In [9]:
## the date features are extremely high-correlated 
#train_dat.corr()

In [10]:
start_time = time.time()
tmp_train_date['start_time'] = tmp_train_date[start_time_column_name]
for column in tmp_train_date.columns:
    if column != 'start_time':
        tmp_train_date[column] = tmp_train_date[column] - tmp_train_date['start_time']
print 'finish substract start_time using {} seconds'.format(round(time.time() - start_time, 2))

finish substract start_time using 0.46 seconds


In [11]:
start_time = time.time()
tmp_test_date = test_dat.copy()
tmp_test_date['start_time'] = tmp_test_date[start_time_column_name]
for column in tmp_train_date.columns:
    if column != 'start_time':
        tmp_test_date[column] = tmp_test_date[column] - tmp_test_date['start_time']
print 'finish substract start_time using {} seconds'.format(round(time.time() - start_time, 2))

finish substract start_time using 0.75 seconds


In [12]:
print tmp_test_date.shape
tmp_test_date.head()

(674503, 68)


,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S3_D74,L0_S4_D106,L0_S4_D111,L0_S5_D115,L0_S5_D117,L0_S6_D120,...,L3_S38_D3961,L3_S39_D3966,L3_S43_D4062,L3_S44_D4101,L3_S47_D4140,L3_S49_D4208,L3_S49_D4213,L3_S50_D4242,L3_S51_D4255,start_time
Id,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.01,NaN,NaN,0.03,0.03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255.45
10,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.03,0.03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,907.34
12,0.0,0.0,NaN,0.0,0.0,0.02,0.02,NaN,NaN,NaN,...,20.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,602.64
15,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.01,0.01,0.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,575.68
20,0.0,0.0,0.01,NaN,NaN,0.02,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271.42


In [13]:
print tmp_train_date.shape
tmp_train_date.head()

(673861, 68)


,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S3_D74,L0_S4_D106,L0_S4_D111,L0_S5_D115,L0_S5_D117,L0_S6_D120,...,L3_S38_D3961,L3_S39_D3966,L3_S43_D4062,L3_S44_D4101,L3_S47_D4140,L3_S49_D4208,L3_S49_D4213,L3_S50_D4242,L3_S51_D4255,start_time
Id,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.00,NaN,NaN,0.02,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.24
7,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.02,0.02,0.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1618.70
9,0.0,0.0,0.01,NaN,NaN,0.02,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1149.20
11,0.0,0.0,NaN,0.00,0.00,0.02,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,602.64
13,0.0,0.0,NaN,0.01,0.01,0.02,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1331.66


In [14]:
print tmp_test_date.min().min(), tmp_train_date.max().max()
print tmp_train_date.min().min(), tmp_train_date.max().max()

0.0 1713.71


In [16]:
tmp_train_date = tmp_train_date.fillna(-1.)
tmp_test_date = tmp_test_date.fillna(-1.)

In [19]:
tmp_train_date.dtypes.value_counts()

float64    68
dtype: int64

### encode the numerical data

In [36]:
tmp_num_train = train_num.copy()
tmp_num_test = test_num.copy()

In [37]:
tmp_num_train.head()

,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,...,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S48_F4196,L3_S48_F4198,L3_S49_F4211,L3_S49_F4226,L3_S50_F4243,L3_S50_F4253,Response
Id,,,,,,,,,,,,,,,,,,,,,
4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,-0.046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,-0.099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
13,0.003,0.019,0.294,0.312,0.031,0.161,0.022,0.088,-0.005,-0.003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [38]:
print tmp_num_train.min().min(), tmp_num_train.max().max()
print tmp_num_test.min().min(), tmp_num_test.max().max()

-0.984 1.0
-0.984 1.0


In [39]:
tmp_num_train = tmp_num_train.fillna(-1.)
tmp_num_test = tmp_num_test.fillna(-1.)

In [40]:
tmp_num_train.head()

,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,...,L3_S47_F4153,L3_S47_F4158,L3_S47_F4163,L3_S48_F4196,L3_S48_F4198,L3_S49_F4211,L3_S49_F4226,L3_S50_F4243,L3_S50_F4253,Response
Id,,,,,,,,,,,,,,,,,,,,,
4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,-0.046,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,-0.099,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
13,0.003,0.019,0.294,0.312,0.031,0.161,0.022,0.088,-0.005,-0.003,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


### process the categorical features

In [33]:
start_time = time.time()
combined_cat = pd.concat([train_cat, test_cat])
## convert to string so that column is categorical
combined_cat = combined_cat.astype(str)
## One-Hot encode all the categorical columns
oneHot_combined_cat = pd.get_dummies(combined_cat, dummy_na=True)

train_index = train_cat.index
test_index  = test_cat.index
oneHot_train_cat = oneHot_combined_cat.ix[train_index]
oneHot_test_cat  = oneHot_combined_cat.ix[test_index]
print 'finish OneHot encoding the categorical columns, using {} seconds'.format(round(time.time() - start_time, 2))

finish OneHot encoding the categorical columns, using 11.69 seconds


In [35]:
print oneHot_train_cat.shape
oneHot_train_cat.head()

(673861, 77)


,L2_S26_F3038_1.0,L2_S26_F3038_nan,L2_S26_F3038_nan,L2_S27_F3131_1.0,L2_S27_F3131_nan,L2_S27_F3131_nan,L3_S29_F3317_1.0,L3_S29_F3317_nan,L3_S29_F3317_nan,L3_S29_F3475_1.0,...,L3_S35_F3912_nan,L3_S35_F3912_nan,L3_S49_F4217_1.0,L3_S49_F4217_nan,L3_S49_F4217_nan,L3_S49_F4220_16.0,L3_S49_F4220_2.0,L3_S49_F4220_4.0,L3_S49_F4220_nan,L3_S49_F4220_nan
Id,,,,,,,,,,,,,,,,,,,,,
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
combined_train = pd.concat([oneHot_train_cat, tmp_num_train, tmp_train_date], axis=1)
combined_test  = pd.concat([oneHot_test_cat,  tmp_num_test,  tmp_test_date],  axis=1)

In [46]:
print combined_train.shape, combined_test.shape

(673861, 437) (674503, 436)


In [48]:
print combined_test.isnull().sum().sum(), '\n \n', combined_train.isnull().sum().sum()

0 0


In [51]:
print combined_test.dtypes.value_counts(), '\n \n',  combined_train.dtypes.value_counts()

float64    436
dtype: int64 
 
float64    436
int64        1
dtype: int64


In [3]:
start_time = time.time()
combined_train.to_csv('bosch_processed_regular_filled_thres_10_train_data.csv')
combined_test.to_csv('bosch_processed_regular_filled_thres_10_test_data.csv')
print 'saving data using {} seconds'.format(round(time.time() - start_time, 2))

NameError: name 'time' is not defined